In [1]:
# from scouting_detector import *
%load_ext autoreload
%autoreload 2
import importlib
import sc2reader
from sc2reader.engine.plugins import SelectionTracker, APMTracker
from selection_plugin import ActiveSelection
sc2reader.engine.register_plugin(SelectionTracker())
sc2reader.engine.register_plugin(APMTracker())
sc2reader.engine.register_plugin(ActiveSelection())

In [2]:
with open("valid_game_ids.txt") as fp:
    replay_files = [f"replays/{x.strip()}" for x in fp.readlines()][:10]

In [3]:
import logging, sys
root = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [6]:
rs = list(sc2reader.load_replays(replay_files))

2020-08-12 14:10:58,371 - sc2reader.events.game.CommandEvent - ERROR - replays/spawningtool_59347.SC2Replay
2020-08-12 14:10:58,371 - sc2reader.events.game.CommandEvent - ERROR - Release String: 4.12.0.80188
2020-08-12 14:10:58,372 - sc2reader.events.game.CommandEvent - ERROR - 	Player 1 - Stats (Protoss)
2020-08-12 14:10:58,372 - sc2reader.events.game.CommandEvent - ERROR - 	Player 2 - SpeCial (Terran)
2020-08-12 14:10:58,372 - sc2reader.engine.plugins.context.ContextLoader - ERROR - 15039	Stats	Missing ability 56E0 from Build
2020-08-12 14:10:58,373 - sc2reader.engine.plugins.context.ContextLoader - ERROR - 695 0
2020-08-12 14:10:58,832 - sc2reader.data.Build - ERROR - Unable to change type of None [0] to 888 [frame 1295]; unit type not found in build 77379
2020-08-12 14:10:58,836 - sc2reader.events.game.CommandEvent - ERROR - replays/spawningtool_59329.SC2Replay
2020-08-12 14:10:58,836 - sc2reader.events.game.CommandEvent - ERROR - Release String: 4.12.0.80188
2020-08-12 14:10:58,83

In [7]:
print([f"has build {r.build}, but best available datapack is {r.datapack.id}" for r in rs if hasattr(r, "marked_error") and r.marked_error])
errors = [r for r in rs if (hasattr(r, "marked_error") and r.marked_error)]
rs = [r for r in rs if not (hasattr(r, "marked_error") and r.marked_error)]
print(f"{len(rs)} good replays: {[r.filename for r in rs]}")

['has build 80188, but best available datapack is 77379', 'has build 80188, but best available datapack is 77379', 'has build 80188, but best available datapack is 77379', 'has build 80188, but best available datapack is 77379', 'has build 80188, but best available datapack is 77379']
5 good replays: ['replays/spawningtool_59334.SC2Replay', 'replays/spawningtool_59330.SC2Replay', 'replays/spawningtool_59303.SC2Replay', 'replays/spawningtool_59259.SC2Replay', 'replays/spawningtool_59258.SC2Replay']


In [ ]:
import sc2reader.data as sc2data
sc2data.ABIL_LOOKUP

In [12]:
for err in errors:
    print()
    print(err.filename)
    for e in err.events:
        if hasattr(e, "ability_id") and e.ability_id not in err.datapack.abilities:
            print(e.active_selection, e.ability_type, e.ability_type_data, e.ability_link, e.command_index)


replays/spawningtool_59347.SC2Replay
[Nexus [59C0001]] TargetUnit {'flags': 111, 'timer': 0, 'unit_tag': 110624769, 'unit_link': 468, 'control_player_id': 1, 'upkeep_player_id': 1, 'point': {'x': 188416, 'y': 389120, 'z': 40896}} 695 0

replays/spawningtool_59329.SC2Replay
[BarracksFlying [30C0001]] None None 147 30

replays/spawningtool_59300.SC2Replay
[Nexus [2CC0001], Nexus [37C0001], Nexus [50C0002], Nexus [6980002]] TargetUnit {'flags': 111, 'timer': 0, 'unit_tag': 116654082, 'unit_link': 468, 'control_player_id': 1, 'upkeep_player_id': 1, 'point': {'x': 221184, 'y': 471040, 'z': 40944}} 695 0

replays/spawningtool_59281.SC2Replay
[Nexus [3540001], Nexus [3E00001], Nexus [48C0001]] TargetUnit {'flags': 111, 'timer': 0, 'unit_tag': 86769665, 'unit_link': 468, 'control_player_id': 2, 'upkeep_player_id': 2, 'point': {'x': 458752, 'y': 548864, 'z': 40912}} 695 0
[Nexus [3540001], Nexus [3E00001], Nexus [48C0001]] TargetPoint {'point': {'x': 190183, 'y': 248422, 'z': 42006}} 719 0
[Ne

In [26]:
print([(e.name, e.ability_name, e.active_selection, e.ability_link) for e in err.events if 
 hasattr(e, "ability_name") and "Lift" in e.ability_name])
[u for u in err.objects.values() if u.name and "Flying" in u.name]

[('BasicCommandEvent', 'LiftBarracks', [BarracksFlying [3B80001]], 148), ('BasicCommandEvent', 'LiftStarport', [Starport [4500001]], 152), ('BasicCommandEvent', 'LiftBarracks', [BarracksFlying [3B80001]], 148), ('BasicCommandEvent', 'LiftCommandCenter', [OrbitalCommand [4C00001]], 145), ('BasicCommandEvent', 'LiftStarport', [Starport [4500001]], 152), ('BasicCommandEvent', 'LiftBarracks', [BarracksFlying [3B80001]], 148), ('BasicCommandEvent', 'LiftBarracks', [Barracks [3B40001]], 148), ('BasicCommandEvent', 'LiftStarport', [Starport [44C0001]], 152), ('BasicCommandEvent', 'LiftOrbitalCommand', [OrbitalCommand [4E40001]], 230), ('BasicCommandEvent', 'LiftBarracks', [Barracks [5640001]], 148), ('BasicCommandEvent', 'LiftBarracks', [BarracksFlying [3B80001]], 148), ('BasicCommandEvent', 'LiftFactory', [Factory [6100001]], 150), ('BasicCommandEvent', 'LiftBarracks', [Barracks [6440001]], 148), ('BasicCommandEvent', 'LiftCommandCenter', [OrbitalCommand [3580001], OrbitalCommand [4180001], 

[BarracksFlying [3B80001],
 OrbitalCommandFlying [4140001],
 OrbitalCommandFlying [9100003],
 OrbitalCommandFlying [1C0002],
 OrbitalCommandFlying [8E40002],
 CommandCenterFlying [9180016]]

In [27]:
r = rs[0]
[(o.owner, o.location, o.started_at, o.finished_at, o.died_at) for o in r.objects.values() if "Command" in o.name and o.owner]

[(Player 1 - Beastyqt (Terran), (160, 46), 0, 0, None),
 (Player 2 - demuLarva (Terran), (55, 157), 0, 0, None),
 (Player 2 - demuLarva (Terran), (82, 160), 2353, 3953, None),
 (Player 1 - Beastyqt (Terran), (133, 43), 2435, 4035, None),
 (Player 2 - demuLarva (Terran), (71, 131), 5776, 7376, None),
 (Player 1 - Beastyqt (Terran), (103, 44), 6238, 7838, None),
 (Player 2 - demuLarva (Terran), (86, 153), 11897, 13497, None),
 (Player 2 - demuLarva (Terran), (58, 141), 13413, 15013, None),
 (Player 2 - demuLarva (Terran), (85, 153), 13723, 15323, None),
 (Player 1 - Beastyqt (Terran), (167, 58), 13967, 15567, None),
 (Player 1 - Beastyqt (Terran), (144, 72), 14111, 15711, None),
 (Player 1 - Beastyqt (Terran), (144, 72), 16451, None, 17502),
 (Player 1 - Beastyqt (Terran), (110, 53), 17161, 18761, None),
 (Player 2 - demuLarva (Terran), (85, 129), 20416, 22016, None),
 (Player 1 - Beastyqt (Terran), (136, 78), 23736, 25336, None)]

In [ ]:
es = [e for e in r.events if isinstance(e, sc2reader.events.game.TargetUnitCommandEvent) or 
      isinstance(e, sc2reader.events.game.TargetPointCommandEvent)]

In [ ]:
[(e.name, e.ability_name) for e in es if "Command" in e.ability_name]
# [(e.target_unit_id, event.location) for e in es if "Command" in e.ability_name]

In [ ]:
sorted([(u.location, u) for u in r.objects.values() if "Mineral" in u.name and hasattr(u, "location")])

In [28]:
from xml.dom import minidom
r.load_map()
xmldoc = minidom.parseString(r.map.archive.read_file('Objects'))
itemlist = xmldoc.getElementsByTagName('ObjectUnit')
mineralPosStrs = [ou.attributes['Position'].value for ou in itemlist if 'MineralField' in ou.attributes['UnitType'].value]
mineralLocs = [tuple([float(num) for num in mps.split(',')[0:2]]) for mps in mineralPosStrs]
sorted(mineralLocs)

[(42.0, 115.5),
 (43.0, 114.5),
 (43.0, 116.5),
 (43.0, 118.5),
 (44.0, 119.5),
 (47.0, 120.5),
 (48.0, 158.5),
 (49.0, 121.5),
 (49.0, 157.5),
 (49.0, 159.5),
 (49.0, 161.5),
 (50.0, 120.5),
 (50.0, 162.5),
 (53.0, 163.5),
 (55.0, 42.5),
 (55.0, 164.5),
 (56.0, 41.5),
 (56.0, 163.5),
 (58.0, 71.5),
 (58.0, 73.5),
 (59.0, 40.5),
 (59.0, 68.5),
 (59.0, 70.5),
 (59.0, 74.5),
 (59.0, 76.5),
 (60.0, 39.5),
 (60.0, 67.5),
 (60.0, 77.5),
 (63.0, 39.5),
 (64.0, 40.5),
 (64.0, 133.5),
 (65.0, 131.5),
 (65.0, 133.5),
 (65.0, 135.5),
 (66.0, 136.5),
 (67.0, 41.5),
 (68.0, 42.5),
 (69.0, 137.5),
 (71.0, 138.5),
 (72.0, 137.5),
 (75.0, 161.5),
 (76.0, 160.5),
 (76.0, 162.5),
 (76.0, 164.5),
 (77.0, 165.5),
 (80.0, 166.5),
 (82.0, 167.5),
 (83.0, 166.5),
 (88.0, 70.5),
 (89.0, 67.5),
 (89.0, 69.5),
 (89.0, 71.5),
 (90.0, 66.5),
 (93.0, 65.5),
 (95.0, 64.5),
 (96.0, 65.5),
 (103.0, 38.5),
 (104.0, 37.5),
 (105.0, 160.5),
 (106.0, 38.5),
 (106.0, 159.5),
 (106.0, 161.5),
 (106.0, 163.5),
 (107.0, 164

In [40]:
landOCCC = ['LandOrbitalCommand', 'LandCommandCenter']
OCCC_names = ['OrbitalCommand', 'CommandCenter']
tier1_base_names = ["Hatchery", "Nexus", "CommandCenter"]
base_names = set(["Hatchery", "Lair", "Hive", "Nexus", "CommandCenter", "CommandCenterFlying", "OrbitalCommand", "OrbitalCommandFlying","PlanetaryFortress"])
build_base_names = ['BuildHatchery','BuildNexus','BuildCommandCenter']

efilter = lambda e: (e.name == 'TargetPointCommandEvent' and e.ability_name in landOCCC) or \
          (e.name == 'UnitTypeChangeEvent' and e.unit_type_name in OCCC_names) or \
          (e.name in ['UnitDoneEvent', 'UnitInitEvent'] and e.unit is not None and e.unit.name in base_names) or \
          (e.name == "TargetPointCommandEvent" and e.ability_name in build_base_names)

for r in rs:
    print(f"\n{r.filename}")
    for event in filter(efilter, r.events):
    #     if hasattr(event, 'unit') and event.unit.owner is None:
    #       # crazy, but some misparsed replays have events with units without a known owner
    #       continue
        if event.name == 'TargetPointCommandEvent':
            if event.ability_name in landOCCC:
                selecteds = event.active_selection
                selected_occcs = [obj for obj in selecteds if sorted(obj.type_history.items())[0][1].name == 'CommandCenter']
                if len(selected_occcs) == 0:
                    print(f"Landing an OC/CC but none selected. WTF. event={event}, selecteds={selecteds}")
                else:
                    selected_occc = selected_occcs[0]
                    print(selected_occcs)
                    print(f"{event.player} landing {selected_occc} at {event.frame} at {event.location} (finished at {selected_occc.finished_at})")
            elif event.ability_name in build_base_names:
                x,y,z = event.location
                print(f"{event.player} ordered {event.ability_name} at {framestr(event.frame)} {(x,y)}")
                base_type = event.ability_name[5:]
                matched_to_unitinit = False
                # sometimes the UnitInit tracker event comes in the same
                # frame as the build command
                for base in event.player.bases:
                    if base.started_at == event.frame and (base in baselocations) and abs(baselocations[base][0] - x) < TRACKER_GRID_SIZE and abs(baselocations[base][1] - y) < TRACKER_GRID_SIZE:
                        if debug_miningbases and debuglevel_miningbases >= 1:
                            print "Matched to {}".format(base)
                        baselocations[base] = (x,y)
                        matched_to_unitinit = True
                    elif debug_miningbases and debuglevel_miningbases >= 1:
                        print "Didnt match. Base: {} {} {}, event {} {} {}".format(base.started_at, base.location[0], base.location[1], event.frame, x, y)
                if not matched_to_unitinit:
                    for index, (frame1, (x1,y1), owner) in enumerate(ordered_bases[base_type]):
                        if (abs(x-x1) < BUILDING_SIZE and abs(y-y1) < BUILDING_SIZE):
                            if debug_miningbases and debuglevel_miningbases >= 1:
                                print "Same location replaces a previous command"
                            ordered_bases[base_type][index] = (event.frame, (x,y), event.player)
                            break
                    else:
                        if debug_miningbases and debuglevel_miningbases >= 1:
                            print "No replacement was found"
                        ordered_bases[base_type].append( (event.frame, (x,y), event.player) )


replays/spawningtool_59334.SC2Replay
[OrbitalCommand [4CC0003]]
Player 2 - demuLarva (Terran) landing OrbitalCommand [4CC0003] at 8630 at (71.5, 131.5, 40892) (finished at 7376)
[PlanetaryFortress [72C0001]]
Player 1 - Beastyqt (Terran) landing PlanetaryFortress [72C0001] at 13386 at (95.5, 71.5, 40894) (finished at 13368)
[OrbitalCommand [5900004]]
Player 2 - demuLarva (Terran) landing OrbitalCommand [5900004] at 13679 at (112.5, 159.5, 32705) (finished at 13497)
[PlanetaryFortress [8540001]]
Player 1 - Beastyqt (Terran) landing PlanetaryFortress [8540001] at 16995 at (61.5, 46.5, 40902) (finished at 16930)
[OrbitalCommand [7EC0001]]
Player 1 - Beastyqt (Terran) landing OrbitalCommand [7EC0001] at 18121 at (144.5, 72.5, 40892) (finished at 15711)
[PlanetaryFortress [7E80002]]
Player 2 - demuLarva (Terran) landing PlanetaryFortress [7E80002] at 18239 at (120.5, 132.5, 40895) (finished at 17688)
[PlanetaryFortress [7E80002]]
Player 2 - demuLarva (Terran) landing PlanetaryFortress [7E80

In [ ]:
from math import sqrt
import numpy as np
def dist(loc1, loc2):
    return sqrt((loc1[0] - loc2[0]) ** 2 + (loc1[1] - loc2[1]) ** 2)

# see your journal entry 20130528 for how this value got set
MINING_BASE_MIN_DIST = 70**0.5


def isMiningLoc(mineralLocs, location):
    # on every melee map so far, this has been the limit for the
    # sixth-closest mineral patch from the starting base, in squared
    # distance units.  Its as large as it is due to discretization
    # effects, since locations are reported on a grid 4x coarser than
    # the actual grid units.

    closeMineralLocsAndDists = [(mineralLoc,dist(mineralLoc, location)) for mineralLoc in mineralLocs \
                                if sqdist(mineralLoc, location) <= MINING_BASE_MIN_DIST]

    # debug/research purposes only
    closeMineralLocsAndDists = sorted(closeMineralLocsAndDists, key=lambda pair: pair[1])

    if len(closeMineralLocsAndDists) < 6:
        print(sorted([sqdist(mineralLoc, location) for mineralLoc in mineralLocs]))
        return False
    return True

In [95]:
from math import sqrt

def dist(loc1, loc2):
    return sqrt((loc1[0] - loc2[0]) ** 2 + (loc1[1] - loc2[1]) ** 2)

# see your journal entry 20130528 for how this value got set
MINING_BASE_MIN_DIST = 70**0.5

def isMiningLoc(mineralLocs, location):
    # on every melee map so far, this has been the limit for the
    # sixth-closest mineral patch from the starting base, in squared
    # distance units.  Its as large as it is due to discretization
    # effects, since locations are reported on a grid 4x coarser than
    # the actual grid units.

    closeMineralLocsAndDists = [(mineralLoc,dist(mineralLoc, location)) for mineralLoc in mineralLocs \
                                if dist(mineralLoc, location) <= MINING_BASE_MIN_DIST]

    # debug/research purposes only
    closeMineralLocsAndDists = sorted(closeMineralLocsAndDists, key=lambda pair: pair[1])

    if len(closeMineralLocsAndDists) < 6:
#         print(sorted([round(dist(mineralLoc, location), 1) for mineralLoc in mineralLocs]))
        return False
    return True

def get_current_name(unit, frame):
    name = unit.name
    for f, t in unit.type_history.items():
        if frame >= f:
            name = t.name
    return name

for r in rs:
#     r.load_map()
#     xmldoc = minidom.parseString(r.map.archive.read_file('Objects'))
#     itemlist = xmldoc.getElementsByTagName('ObjectUnit')
#     mineralPosStrs = [ou.attributes['Position'].value for ou in itemlist if 'MineralField' in ou.attributes['UnitType'].value]
#     mineralLocs = [tuple([float(num) for num in mps.split(',')[0:2]]) for mps in mineralPosStrs]
    mineralLocs = [o.location for o in r.objects.values() if o.name and "Mineral" in o.name and hasattr(o, "location")]
    under_construction = {p.team_id: {} for p in r.humans if p not in r.observers}
    starting_bases = [(e.unit, e.location) for e in r.events if e.frame == 0 
                      and isinstance(e, sc2reader.events.UnitBornEvent) 
                      and sorted(e.unit.type_history.items())[0][1].name in tier1_base_names]
    completed = {p.team_id: {u.id: l for u, l in starting_bases if u.owner == p} for p in r.humans if p not in r.observers}
    assert len(completed) == 2 and all(len(v) == 1 for v in completed.values())
    
    print(f"\n{r.filename}")
    for e in r.events:
        if isinstance(e, sc2reader.events.UnitInitEvent) and get_current_name(e.unit, e.frame) in tier1_base_names:
            name = get_current_name(e.unit, e.frame)
            print(f"{e.frame} BUILDING {e.unit} ({name}) AT {e.location} {isMiningLoc(mineralLocs, e.location)}")
            assert e.unit.id not in under_construction[e.unit.owner.team_id]
            assert e.unit.id not in completed[e.unit.owner.team_id]
            under_construction[e.unit.owner.team_id][e.unit.id] = e.location
        if isinstance(e, sc2reader.events.UnitDoneEvent) and get_current_name(e.unit, e.frame) in tier1_base_names:
            name = get_current_name(e.unit, e.frame)
            assert e.unit.id in under_construction[e.unit.owner.team_id]
            assert e.unit.id not in completed[e.unit.owner.team_id]
            location = under_construction[e.unit.owner.team_id][e.unit.id]
            completed[e.unit.owner.team_id][e.unit.id] = location
            del under_construction[e.unit.owner.team_id][e.unit.id]
            print(f"{e.frame} COMPLETED {e.unit} ({name}) AT {location}")
        if e.name == 'TargetPointCommandEvent':
            if e.ability_name in landOCCC:
                selecteds = e.active_selection
                selected_occcs = [obj for obj in selecteds if sorted(obj.type_history.items())[0][1].name == 'CommandCenter']
                if len(selected_occcs) == 0:
                    print(f"Landing an OC/CC but none selected. WTF. event={event}, selecteds={selecteds}")
                else:
                    name = selected_occcs[0].name
                    for f, t in selected_occcs[0].type_history.items():
                        if e.frame > f:
                            name = t.name
                    print(f"{e.frame} LANDED {selected_occcs[0]} ({name}) AT {e.location} {isMiningLoc(mineralLocs, e.location)}")
            


replays/spawningtool_59334.SC2Replay
2353 BUILDING OrbitalCommand [3D40001] (CommandCenter) AT (82, 160) True
2435 BUILDING OrbitalCommand [3D80001] (CommandCenter) AT (133, 43) True
3953 COMPLETED OrbitalCommand [3D40001] (CommandCenter) AT (82, 160)
4035 COMPLETED OrbitalCommand [3D80001] (CommandCenter) AT (133, 43)
5776 BUILDING OrbitalCommand [4CC0003] (CommandCenter) AT (62, 143) False
6238 BUILDING OrbitalCommand [5000001] (CommandCenter) AT (103, 44) True
7376 COMPLETED OrbitalCommand [4CC0003] (CommandCenter) AT (62, 143)
7838 COMPLETED OrbitalCommand [5000001] (CommandCenter) AT (103, 44)
8630 LANDED OrbitalCommand [4CC0003] (OrbitalCommandFlying) AT (71.5, 131.5, 40892) True
11768 BUILDING PlanetaryFortress [72C0001] (CommandCenter) AT (133, 50) False
11897 BUILDING OrbitalCommand [5900004] (CommandCenter) AT (86, 153) False
13368 COMPLETED PlanetaryFortress [72C0001] (CommandCenter) AT (133, 50)
13386 LANDED PlanetaryFortress [72C0001] (CommandCenterFlying) AT (95.5, 71.5,

In [70]:
print(70**0.5)
for r in rs:
    print(f"\n{r.filename}")
    mineralLocs = [o.location for o in r.objects.values() if o.name and "Mineral" in o.name and hasattr(o, "location")]
    starting_bases = [o for o in r.objects.values() if o.name and sorted(o.type_history.items())[0][1].name in tier1_base_names and o.started_at == 0]
    for b in starting_bases:
        print(b, b.owner)
        print(sorted([round(dist(b.location, loc), 1) for loc in mineralLocs]))

8.366600265340756

replays/spawningtool_59334.SC2Replay
OrbitalCommand [31C0001] Player 1 - Beastyqt (Terran)
[6.0, 6.7, 7.0, 7.1, 7.3, 7.8, 8.1, 8.1, 19.4, 20.2, 20.6, 21.2, 22.5, 23.3, 23.8, 23.9, 24.2, 25.3, 25.6, 25.6, 25.6, 27.5, 27.9, 28.5, 36.7, 37.5, 38.1, 39.8, 41.1, 43.0, 44.3, 44.9, 49.1, 50.0, 50.2, 50.4, 51.5, 54.6, 56.7, 57.6, 66.8, 67.4, 69.6, 72.8, 74.0, 74.6, 75.3, 75.9, 80.1, 81.1, 83.1, 84.0, 86.0, 87.1, 89.1, 90.1, 92.1, 92.1, 92.7, 93.1, 94.0, 95.9, 96.2, 97.1, 97.3, 99.2, 100.2, 100.3, 100.8, 101.2, 102.2, 103.4, 103.8, 104.1, 104.7, 104.8, 105.0, 105.1, 105.4, 105.5, 115.0, 115.6, 116.0, 116.5, 117.0, 117.3, 118.1, 118.2, 125.2, 126.6, 126.6, 127.0, 127.5, 127.9, 128.0, 128.7, 128.8, 128.9, 129.1, 129.2, 129.4, 129.6, 130.1, 130.2, 132.6, 134.0, 135.1, 135.3, 136.3, 136.7, 137.1, 137.4, 141.6, 142.6, 143.0, 143.2, 144.0, 144.2, 144.8, 145.1, 156.5, 157.0, 158.0, 158.4, 158.4, 158.5, 159.8, 159.9]
OrbitalCommand [3500001] Player 2 - demuLarva (Terran)
[6.0, 6.1, 6

In [81]:
[(e.unit, e.location) for e in r.events if e.frame == 0 and "UnitBorn" in e.name and "Command" in e.unit.name]

[(OrbitalCommand [3400001], (142, 139))]